In [103]:
#########Generate customers and vechicles 
import numpy as np
import math
N=20
loc_x = np.random.uniform(0,100,N)
loc_y = np.random.uniform(0,100,N)
#print(loc_x)
#print(loc_y)
#calculate distance
distance = np.zeros((N,N))
for i in range(N):
    for j in range(N):
        dist = math.sqrt((loc_x[i]-loc_x[j])**2+(loc_y[i]-loc_y[j])**2)
        distance[i][j] = dist
#print(distance)
dist=distance.tolist()
Q=20
M=4
demand=np.random.uniform(0,8,N)
#demand[0]=0
dem=demand.tolist()
cap=[Q]*M
d=demand

In [104]:
######Primary Route and label the customer in N ways
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] =dist
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

def get_routes(solution, routing, manager):
    
    routes = []
    for route_nbr in range(routing.vehicles()):
        
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
            routes.append(route)
    return routes


In [105]:
testdata=[]

In [106]:
 for i in range(0,N):
    
    def create_data_model():
        data = {}
        data['distance_matrix'] =dist
        data['num_vehicles'] = 1
        data['depot'] = i
        return data
    
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)
        
    routes = get_routes(solution, routing, manager)
    temp = list(set(tuple(x) for x in routes))
    testdata.append(temp)

Objective: 404 miles
Route for vehicle 0:
 0 -> 1 -> 17 -> 4 -> 5 -> 14 -> 12 -> 7 -> 10 -> 18 -> 15 -> 2 -> 13 -> 8 -> 6 -> 19 -> 16 -> 9 -> 3 -> 11 -> 0

Objective: 415 miles
Route for vehicle 0:
 1 -> 0 -> 5 -> 14 -> 6 -> 11 -> 3 -> 9 -> 16 -> 19 -> 8 -> 13 -> 2 -> 15 -> 18 -> 10 -> 7 -> 12 -> 4 -> 17 -> 1

Objective: 401 miles
Route for vehicle 0:
 2 -> 13 -> 8 -> 19 -> 16 -> 9 -> 3 -> 11 -> 0 -> 1 -> 17 -> 12 -> 4 -> 5 -> 14 -> 6 -> 7 -> 10 -> 18 -> 15 -> 2

Objective: 401 miles
Route for vehicle 0:
 3 -> 11 -> 0 -> 1 -> 17 -> 12 -> 4 -> 5 -> 14 -> 6 -> 7 -> 10 -> 18 -> 15 -> 2 -> 13 -> 8 -> 19 -> 16 -> 9 -> 3

Objective: 412 miles
Route for vehicle 0:
 4 -> 5 -> 0 -> 1 -> 17 -> 12 -> 7 -> 10 -> 18 -> 15 -> 2 -> 13 -> 8 -> 9 -> 3 -> 11 -> 16 -> 19 -> 6 -> 14 -> 4

Objective: 411 miles
Route for vehicle 0:
 5 -> 0 -> 1 -> 17 -> 4 -> 12 -> 7 -> 10 -> 18 -> 15 -> 2 -> 13 -> 8 -> 9 -> 3 -> 11 -> 16 -> 19 -> 6 -> 14 -> 5

Objective: 404 miles
Route for vehicle 0:
 6 -> 19 -> 16 -> 9 ->

In [107]:
new=tuple()
for i in range(len(testdata)):
    temp=testdata[i]
    for j in range(len(temp)):
        new=new+tuple(j for j in temp)
        

In [108]:

testdata=new

In [139]:
#split customers to vechicles 
ans=[testdata[2][i:i + 5] for i in range(0, len(testdata[0]), 5)]
ans=ans[0:5-1]

In [140]:
newd=testdata[2][0:20]
dnew = [d[i] for i in newd]
dnew=np.array(dnew)

In [141]:
len(testdata[2][0:20])

20

In [142]:

n=int(N/M)
s= [None] * M
w= [None] * M
e= [None] * M
ee= [None] * M
cn=np.arange(n)
d=dnew


In [143]:
s[0]=0
w[0]=sum(d[cn])
e[0]=math.ceil(w[0]/Q)*Q-w[0]
cn=cn+n
ee[0]=min(e[0],sum(d[cn]))
for i in range(n):
    temp=d[cn]
    if sum(temp[:i+1])>ee[0]:
        s[1]=i+n
        break

In [144]:
cn=np.arange(n)
for j in range(1,M-1):
    if max(cn) < N-1:
        cn=cn+n
    w[j]=sum(d[cn])-e[j-1]
    if w[j]<0:
        w[j]=0
    e[j]=math.ceil(w[j]/Q)*Q-w[j]
    if max(cn) < N-1:
        cn=cn+n
    ee[j]=min(e[j],sum(d[cn]))
  
    temp=d[cn]
    start=(j+1)*n
    end=start+n-1
    res=np.arange(start,end+1,1)
    for i in range(n):
        if sum(temp[:i+1])>ee[j]:
            s[j+1]=res[i]
            break
    

In [145]:
print(e)
print(ee)
print(w)
print(s)

[11.480485537753818, 4.925324655944564, 8.775392729823345, None]
[11.480485537753818, 4.925324655944564, 8.775392729823345, None]
[28.519514462246182, 15.074675344055436, 11.224607270176655, None]
[0, 7, 10, 18]


In [146]:
###########fill the last element###########
end=N
start=N-n
#print(d[start:N])
w[-1]=sum(d[start:end])-e[-2]
if w[-1] <0:
    w[-1]=0
e[-1]=math.ceil(w[-1]/Q)*Q-w[-1]
ee[-1]=0

In [147]:
#######get dd
dd=[None]*N
for j in range(M):
    if j==0:
        dd[0]=d[s[0]]
    else:
        if e[j-1]==0:
            dd[s[j]]=d[s[j]]
        else:
            start=j*n
            end=s[j]+1
            dd[s[j]]=sum(d[start:end])-ee[j-1]

In [148]:
##########insert the depot position and recalculate the dist matrix
test=loc_x
test2=loc_y
depot=np.array(50)
new_x = np.append (depot, test)
new_y = np.append (depot, test2)
new_x=new_x[1:len(new_x)]
new_y=new_y[1:len(new_y)]

In [149]:
newdist = np.zeros((len(new_x),len(new_y)))
for i in range(len(new_x)):
    for j in range(len(new_y)):
        dist = math.sqrt((new_x[i]-new_x[j])**2+(new_y[i]-new_y[j])**2)
        newdist[i][j] = dist
               

In [150]:
##########generate extended route
def extend(M,n,lst):
    res=[]
    for i in range(M):
        j=i+1
        if i==0:
            a=list(range((j-1)*n,(j+1)*n)) #####index
            res.append(a)
        elif i==M-1:
            a=list(range((j-1)*n,j*n))
            res.append(a)
        else:
            m=j
            nn=j+1
            one=list(range((m-1)*n,m*n))
            two=list(range((nn-1)*n,nn*n))
            out=one+two #######actual labeled customer 
            res.append(out)    
    return res
            
    

In [151]:
get=extend(4,5,ans) 
get

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [15, 16, 17, 18, 19]]

In [152]:
travel=[None]*M
for i in range(M):
    cn=0
    path=max(get[i])
    pos=0
    if w[i]>0:
        cn=dd[s[i]]
        then=s[i]+1      ######find the index of demand 
        cn=cn+d[then]
        pos=s[i]+1
        #print(pos)
        while round(cn,2)< round(w[i]+ee[i],2):
            pos=pos+1
            #print(pos)
            cn=cn+d[pos]
        travel[i]=pos

In [123]:
travel

[7, 12, 16, 19]

In [153]:
#######generate the 

7.374539196297876

In [154]:
w[i]+ee[i]

7.374539196297874